In [2]:
from ase                        import Atoms
from ase.visualize              import view
import matplotlib.pyplot            as plt
import graphdot.kernel.molecular    as gkern
import graphdot
import seaborn                      as sns
import numpy                        as np
import timeit
import pandas                       as pd
import scipy
import sklearn.linear_model
import random

In [3]:
# import data from feti_filtered.dat
positions = []
forces = []

with open('feti_filtered.dat') as feti:
    for line in feti:
        parts = line.split()
        if len(parts) == 3:
            positions.append([])
            forces.append([])
        if len(parts) == 6:
            parts_floats = [float(x) for x in parts]
            positions[-1].append(parts_floats[:3])
            forces[-1].append(parts_floats[3:])

In [4]:
# produce graphs
graphs = []
symbols = 'Fe'*64 + 'Ti'*64
for position_step in positions:
    atoms = Atoms(symbols, position_step, pbc=True)
    graphs.append(graphdot.Graph.from_ase(atoms))

In [5]:
def hyper_to_k_sim(nu, lambda_h):
    zeta = 1
    s = 1
    # try higher q for shorter mean path lengths
    q = .2
    # use kernel from atomization paper
    tang_kernel = gkern.Tang2019MolecularKernel(
                        stopping_probability=q,
                        starting_probability=lambda x, y: s,
                        element_prior=nu,
                        edge_length_scale=lambda_h)
    k = tang_kernel(graphs[:50])
    d = np.diag(k)**-0.5
    k_sim = np.diag(d).dot(k).dot(np.diag(d))
    
    return k_sim

In [6]:
def k_sim_to_variance(k_sim):
    flat = k_sim.flatten()
    return np.var(flat)

def k_sim_to_mean(k_sim):
    flat = k_sim.flatten()
    return np.mean(flat)

In [ ]:
nu_s     = np.array([.01,.05,.1,5,.99])
lambda_s = np.array([.01,.05,.1,.5,1])

k_sim_matrix = []
var_matrix = []
mean_matrix=[]
for nu in nu_s:
    k_sim_matrix.append([])
    var_matrix.append([])
    mean_matrix.append([])
    for lambda_h in lambda_s:
        k_sim = hyper_to_k_sim(nu, lambda_h)
        k_sim_matrix[-1].append(k_sim)
        flat = k_sim.flatten()
        var = np.var(flat)
        var_matrix[-1].append(var)
        mean= np.mean(flat)
        mean_matrix[-1].append(mean)
        
sns.heatmap(mean_matrix, square=True,cmap="YlGnBu")

In [ ]:

pd.Series(k_sim_matrix[2][2].flatten()).plot(kind='hist', bins=100)

In [ ]:
mean_matrix


In [ ]:
1-np.array(mean_matrix)

In [ ]:
flat = k_sim.flatten()

In [ ]:
pd.Series(flat).plot(kind='hist', bins=100)

In [ ]:
pd.Series(1-flat).plot(kind='hist', bins=100)

In [ ]:
scipy.stats.gamma.fit(1-flat)

In [ ]:
x = np.linspace(0,.006,100)
plt.plot(x, scipy.stats.gamma.pdf(x, 0.9094693984,loc=0, scale = 0.0010062714))

In [ ]:
dist = scipy.stats.gamma(12.8834, loc=0.99389, scale = 0.0039007)

In [ ]:
#regular points from gamma at .994 to gamma at 1
x = np.linspace(.99,1.5,100)

In [ ]:
plt.plot(x, scipy.stats.gamma.pdf(x, 12.8834,loc=.99389, scale = 0.0039007))

In [ ]:
scipy.stats.gamma(a=5,loc=2, scale = 3).var()

In [ ]:
x = np.linspace(0,10,1000)
plt.plot(x, scipy.stats.gamma.pdf(x, 5, loc=4, scale=4))

In [ ]:
np.var(flat)

In [ ]:
np.mean(flat,axis=0)

In [25]:
def get_partials(hypers, mean0, beta):
    partial_derivatives = []       
    for i, hyper_value in enumerate(hypers):
        delta_hyper = hyper_value/beta
        
        hypers1 = hypers.copy()
        hypers1[i] += delta_hyper
        k_sim1 = hyper_to_k_sim(*hypers1)
        mean1  = k_sim_to_mean(k_sim1)

        change_in_mean = mean1-mean0

        partial_derivatives.append(change_in_mean/delta_hyper)
    return partial_derivatives

In [26]:
# Gradient Ascent to find set of hyperparameters which 
# maximize the dissimilarity of the normalized k matrix

# starting point
# legend: [nu, lambda]
hypers = [.3, .1]

alpha = 10
beta = 100
for iteration in range(10):
    print('Hypers:   ', hypers)

    # calculate 
    k_sim0 = hyper_to_k_sim(*hypers)
    mean0  = k_sim_to_mean(k_sim0)
    print('Mean:     ', mean0)
    
    # produce partial derivatives:
    partial_derivatives = get_partials(hypers, mean0, beta)
    print('Partials: ', partial_derivatives)
    
    # update hypers
    hypers = list(map( (lambda hyp,part:hyp-alpha*part), hypers, partial_derivatives))


Hypers:    [0.3, 0.1]
Mean:      0.9997394233454997
Partials:  [0.00016993842788526337, 0.0015096914496304237]
Hypers:    [0.29830061572114736, 0.08490308550369577]
Mean:      0.9997153003861015
Partials:  [0.00017895404476435487, 0.0016076624499031813]
Hypers:    [0.2965110752735038, 0.06882646100466396]
Mean:      0.9996881325786813
Partials:  [0.0001904775636047991, 0.00170800341691216]
Hypers:    [0.29460629963745577, 0.051746426835542356]
Mean:      0.9996562876466087
Partials:  [0.00021038866459032667, 0.002005848950495389]
Hypers:    [0.2925024129915525, 0.031687937330588466]
Mean:      0.9996075941420437
Partials:  [0.00023352447341865615, 0.0029450183949473936]
Hypers:    [0.29016716825736594, 0.002237753381114531]
Mean:      0.9995006652825931
Partials:  [0.00028324907347860093, 0.003155554825350506]
Hypers:    [0.2873346775225799, -0.02931779487239053]
Mean:      0.9995990502553819
Partials:  [0.0002490275607258972, -0.0032045292379784813]
Hypers:    [0.28484440191532095, 0.

In [7]:
hypers = [.3, .1]
k_sim_0 = hyper_to_k_sim(*hypers)

In [15]:
new_hypers = map(lambda x,y:x+10*y, hypers, partial_derivatives)
list(new_hypers)

[0.30170642506421474, 0.11520199988447324]

In [29]:
graphdot.__version__

'0.5a5'